#### DNN 기반 이진분류 모델 구현
- 데이터셋 : iris.csv
- 피쳐/라벨 : 4개 sepal.length, sepal.width, petal.length, petal.width
- 타겟/라벨 : 1개 Setosa와 나머지 (이진 분류이기에 클래스는 2개)
- 학습 방법 : 지도 학습 > 분류 > 이진분류
- 학습 알고리즘 : 인공신경망 (ANN) -> MLP(Multi Layer Percetron), 심층신경망(DNN)(은닉층 多)
- 프레임 워크 : Pytorch
---
- 모니터링 
    * 기준 : 검정데이터셋의 loss 와 score
    * 평가 : 학습데이터셋의 loss 와 score 와 비교해서 학습 중단여부 결정
    * 선택 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장


[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
# 1. 모델관련
import torch                     # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn            # 인공신경망 관련 모듈
import torch.nn.functional as F  # 손실, 거리 등 함수 관련 모듈

# 2. 데이터 셋 관련                    
from torch.utils.data import DataLoader, Dataset

# 3. 최적화에 관련
import torch.optim as optim      # 최적화 기법 관련 모듈

# 4. 모델 평가
from torchmetrics.classification import F1Score, BinaryF1Score, BinaryConfusionMatrix

# 5. 모델의 구조를 보는 모듈
from torchinfo import summary    # 모델 정보 관련 모듈

# 6. Data 관련
import pandas as pd                 # 데이터 파일 분석 관련 모듈
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하세요~~
print(f'Pytroch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytroch v.2.4.1
Pandas v.2.0.3


In [3]:
# 데이터 로딩
DATA_file ='../data/iris.csv'

# CSV >> DF
irisDF = pd.read_csv(DATA_file)

# 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa


In [4]:
### 타겟 변경 >> 정수화, 클래스 3개 >> 2개로 변경
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [5]:
# 클래스 3개 >> 2개로 변경
irisDF['variety'] = (irisDF['variety'] == 'Setosa')
irisDF['variety'] = irisDF['variety'].astype('int')
irisDF['variety'].unique() , irisDF.head(1)

(array([1, 0]),
    sepal.length  sepal.width  petal.length  petal.width  variety
 0           5.1          3.5           1.4          0.2        1)

In [6]:
### 타겟 정수화 >> Multi 참고
# label = dict(zip(irisDF['variety'].unique().tolist(), range(3)))
# print(f'label => {label}')
# irisDF['variety'] = irisDF['variety'].replace(label)

[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisBCFModel
- 부모 클래스 : nn.Module
- 매개 변수 : 층별 입출력 개수 고정하기 때문에 필요 없음
- 속성 필드 : 
- 기능 역할(필수 메서드) : _ _init_ _() - 모델 구조 설정, forward() - 순방향 학습 <== 오버라이딩(overriding) >> 상속받을 때만 가능
- 클래스 구조
    * 입력층 - 입력  4개 (피쳐 개수) >  출력 10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 - 입력 10개            >  출력  5개 (퍼셉트론 / 뉴런 5개 존재)
    * 출력층 - 입력 5개             >  출력  1개 (퍼셉트론 / 뉴런 1개 존재, 2진분류)

- 활성화함수
    * 클래스 형태 ==> nn.MESLose, nn.ReLU ==> _ _init_ _() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [7]:
class IrisBCMModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(4,10)
        self.hd_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):

        # 입력층 
        y = self.in_layer(input_data)   
        y = F.relu(y)                   # relu => y값의 범위 : 0 <= y

        # 은닉층 : 10개의 숫자 값(y >= 0)
        y=self.hd_layer(y)
        y = F.relu(y)

        # 출력층 : 5개의 숫자 값(y >= 0) >> 2진분류
        return F.sigmoid(self.out_layer(y))

In [8]:
# 모델 인스턴스 생성[확인 용도]
model = IrisBCMModel()
print(model)

IrisBCMModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [9]:
### 모델 사용 메로리 정보 확인
summary(model, input_size=(17,4)) # 데이터양, 피쳐개수

Layer (type:depth-idx)                   Output Shape              Param #
IrisBCMModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 4개
- 타겟 개수 : 1개 [0과 1]
- 클래스 이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성_필드 : featuresDF, targetDF, n_rows, n_features
- 필수메서드
    * __init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __len__(self) : 데이터의 개수 반환
    * __getitme__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [10]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0] # >> 새로 속성 추가, 데이터 수 확인
        self.n_features = featureDF.shape[1] # >> 새로 속성 추가, 피쳐 개수 확인
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
    
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # values하는 이유 >> array를 하기위해
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [11]:
## [테스트] 데이터셋 인스턴스 생성 

# DF에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150,4)
targetDF = irisDF[irisDF.columns[-1:]]  # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

# 데이터 로더 인스턴스 생성
irisDL = DataLoader(irisDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비 <hr>
- 학습 횟수 : EPOCH     << 처음 ~ 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE    << 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE    << 텐서 저장 및 실행 위치(GPU/CPU)
- 학습률 : LR   << 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정, 0.001 ~ 0.1 사이를 많이 줌(값이 작을수록 촘촘하게 이동) >> 하이퍼 파라미터

In [12]:
## 학습 진행 관련 설정
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수 , 성능 지표)

In [13]:
# 모델 인스턴스
model = IrisBCMModel().to(DEVICE)

In [14]:
## DS와 DL 인스턴스
# 학습용, 검증용, 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1, stratify=targetDF)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1, stratify=y_train)

print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{y_train.value_counts()} {y_test.value_counts()} {y_val.value_counts()}')
print(f'{type(y_train)} {type(y_test)} {type(y_val)}')

# 테이터 확인
trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 학습용 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE) ## >> 'drop_last = True' 하면 나머지는 버림, 기본값은 False
                                                      # >> 기본값은 나머지가 생기면 앞에 나열되있던 데이터를 필요한만큼 끌고와 대입입

(84, 4) (38, 4) (28, 4)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
(84, 1) (38, 1) (28, 1)
variety
0          56
1          28
Name: count, dtype: int64 variety
0          25
1          13
Name: count, dtype: int64 variety
0          19
1           9
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


- 최적화, 손실함수 인스턴스 생성

In [15]:
# 최적화 인스턴스 => W,b텐서 즉, model.parameters() 전달 >> W,b 업데이트 시키는이유 >>> 오차를 줄여 최적의 모델을 찾기위해  
optimizer = optim.Adam(model.parameters(), lr=LR)

# 소실함수 인스턴스 => 분류 >> 이진 분류 BinaryCrossEntropyLoss => BCELoss
#                             예측값은 확률값으로 전달 ==> sigmoid() AF처리 후 전달
crossLoss = nn.BCELoss()

[5] 학습 진행 <hr>

In [16]:
# DataLoader로 8.4에서 9로 바꾼다. > 나머지가 남으면 부족한 만큼 앞에 있는 데이터를 가져다가 쓴다
len(trainDL), trainDL.__len__()

(9, 9)

In [17]:
## 학습의 효과 확인, 손실값과 성능평가값 저장 필요 , 검증기능을 
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]
CNT = len(trainDL)

## 학습 모니터링/ 스케줄링 설정 => LOSS_HISTORY, SCORE_HISTORY 활용
# 임계기준(얼마나 봐줄것인가?) : 10번(개발자 재량)제한, 카운팅 해야함
BREAK_CNT=0
LIMIT_VALUE= 10


for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()
    
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = crossLoss(pre_y, targetTS)
        loss_total += loss.item()

        # 성능 평가 계산
        score = BinaryF1Score()(pre_y, targetTS) 
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증 기능
    # 모델 검증 모드 설정 >> 검증이기에 업데이트 불필요
    model.eval() 
    with torch.no_grad():
        
        # 검증 데이터셋
        val_featrueTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론 / 평가 
        pre_val = model(val_featrueTS)
        
        # 손실
        loss_val = crossLoss(pre_val, val_targetTS)
        
        # 성능평가
        score_val = BinaryF1Score()(pre_y, targetTS)

    
    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # score 기준(검증 기준)
    
    if len(SCORE_HISTORY[1]) > 1:
        if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT+=1
            # 현재의 loss값         이전의 loss값
            # 현재 비교하는것은 score이기에 비교했을 때 큰것이 좋다. 이전의 값과 작거나 같은경우 멈출려고한다.
            # 데이터에 따라서 그런 경우가 생길수도 있으니 횟수를 제한을 둬서 값이 반복되거나 커지면 딥러닝 스톱 

               
    # 학습 중단 여부 설정
    if BREAK_CNT> LIMIT_VALUE:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break



성능 및 손실 개선이 없어서 학습 중단


[6] 테스트

In [18]:
# 모델 검증 모드 설정
model.eval()
with torch.no_grad():
    # 검증 데이터셋
    test_featrueTS=torch.FloatTensor(testDS.featureDF.values).to(DEVICE)
    test_targetTS=torch.FloatTensor(testDS.targetDF.values).to(DEVICE)
    
    # 추론/평가
    pre_test=model(test_featrueTS)

    # 손실
    loss_test=crossLoss(pre_test, test_targetTS)
    
    # 성능평가
    # score_val=F1Score(task='binary')(pre_test, val_targetTS)
    score_Test=BinaryF1Score()(pre_test, test_targetTS)
    
print(f'- [TEST] LOSS : {loss_test.item()} SCORE : {score_Test.item()}')

- [TEST] LOSS : 0.691983699798584 SCORE : 0.0


- 모델 저장 방법 <hr>
 + 방법 1 : 모델 파라미터만 저장
 + 방법 2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [19]:
### 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.4787, -0.0906,  0.1323,  0.4248],
                      [-0.3444, -0.2395, -0.4416, -0.3974],
                      [ 0.0709,  0.0625, -0.4098, -0.2615],
                      [ 0.3611, -0.4355, -0.0697, -0.3652],
                      [-0.4723, -0.3114, -0.2050, -0.0777],
                      [-0.0463,  0.4790,  0.2156, -0.3652],
                      [-0.0872, -0.0620, -0.2730,  0.0887],
                      [-0.0062, -0.2623,  0.2467, -0.4629],
                      [ 0.3654,  0.2787, -0.2056,  0.3527],
                      [-0.4681,  0.0066, -0.0727,  0.2808]])),
             ('in_layer.bias',
              tensor([ 0.0715,  0.4921, -0.2246,  0.3694,  0.2403,  0.3322, -0.1847, -0.1566,
                      -0.3504, -0.3988])),
             ('hd_layer.weight',
              tensor([[ 0.0837, -0.0199,  0.0041,  0.1195, -0.1288, -0.1549, -0.1426,  0.0050,
                       -0.2724, -0.1009],
                      [ 0.

- [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [20]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/'

# 저장 파일명
SAVE_FILE = 'model_train_BCF.pth'

# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더/폴더/... 하위 폴더까지 생성

In [21]:
# 모델 저장                                                                                                  
torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)

모델 즉, 가중치와 절편 로딩
- [1] 가중치와 절편 객체로 로딩
- [2] 모델의  state_dict 속성에 저장

In [22]:
# 읽기
wbTS = torch.load(SAVE_PATH+SAVE_FILE, weights_only = True)
wbTS 

OrderedDict([('in_layer.weight',
              tensor([[-0.4787, -0.0906,  0.1323,  0.4248],
                      [-0.3444, -0.2395, -0.4416, -0.3974],
                      [ 0.0709,  0.0625, -0.4098, -0.2615],
                      [ 0.3611, -0.4355, -0.0697, -0.3652],
                      [-0.4723, -0.3114, -0.2050, -0.0777],
                      [-0.0463,  0.4790,  0.2156, -0.3652],
                      [-0.0872, -0.0620, -0.2730,  0.0887],
                      [-0.0062, -0.2623,  0.2467, -0.4629],
                      [ 0.3654,  0.2787, -0.2056,  0.3527],
                      [-0.4681,  0.0066, -0.0727,  0.2808]])),
             ('in_layer.bias',
              tensor([ 0.0715,  0.4921, -0.2246,  0.3694,  0.2403,  0.3322, -0.1847, -0.1566,
                      -0.3504, -0.3988])),
             ('hd_layer.weight',
              tensor([[ 0.0837, -0.0199,  0.0041,  0.1195, -0.1288, -0.1549, -0.1426,  0.0050,
                       -0.2724, -0.1009],
                      [ 0.

In [23]:
# 모델 인트턴스에 저장
model2 = IrisBCMModel() # 층마다 W,b 초기화
model2.load_state_dict(wbTS)

<All keys matched successfully>